In [1]:
import pandas as pd
# import pgeocode
from tqdm.notebook import tqdm
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 100)

In [19]:
test_df = pd.read_csv('/kaggle/input/weights-catboost/test_dataset_test/test/test.csv')

In [4]:
data_account = pd.read_csv('/kaggle/input/hack228/train_dataset_train/train/cntrbtrs.csv', sep=';')
data_transaction = pd.read_csv('/kaggle/input/hack228/train_dataset_train/train/trnsctns.csv', sep=';')

In [20]:
def union_npo(df):
    df = df.drop_duplicates()
    temp = df.groupby(['quarter','npo_account_id'])['client_id'].count()
    temp = temp[temp > 1].reset_index()
    for i in range(temp.shape[0]):
        npo_account_id = temp.iloc[i]['npo_account_id']
        quarter = temp.iloc[i]['quarter']
        df.loc[(df.npo_account_id == npo_account_id) & (df.quarter == quarter), 'churn'] = 1
    df = df.drop_duplicates()
    return df

test_df = union_npo(test_df)

In [ ]:
grouped = data_transaction.drop(columns=['slctn_nmbr', 'year', 'npo_oprtn_date']).groupby(['npo_account_id', 'quarter'])
result = grouped.agg({'npo_sum': ['min', 'max', 'mean', 'count', 'sum'], 
                      'npo_oprtn_grp': ['min', 'max', 'mean', 'count', 'sum']})

result.columns = ['_'.join(col).strip() for col in result.columns.values]
for i in tqdm(range(1, 5)):
    for stat in ['min', 'max', 'mean', 'count', 'sum']:
        result[f'{stat}_npo_sum_{i}_quarters_ago'] = result.groupby('npo_account_id')[f'npo_sum_{stat}'].shift(i)
        result[f'{stat}_npo_oprtn_grp_{i}_quarters_ago'] = result.groupby('npo_account_id')[f'npo_oprtn_grp_{stat}'].shift(i)
test_df = test_df.merge(result, on=['npo_account_id', 'quarter'], how='left')

In [ ]:
# test_df.drop(columns=['npo_sum_max','npo_sum_count',
#                          'npo_sum_sum','npo_oprtn_grp_min','npo_oprtn_grp_max','npo_oprtn_grp_mean',
#                          'npo_oprtn_grp_count','npo_oprtn_grp_sum'], inplace=True)

In [ ]:
test_df[['year', 'quarter_number']] = test_df['quarter'].str.split('Q', expand=True)
test_df['quarter_sin'] = np.sin(2 * np.pi * test_df['quarter_number'].astype(float) / 4)
test_df['quarter_cos'] = np.cos(2 * np.pi * test_df['quarter_number'].astype(float) / 4)

In [ ]:
coordinates_df = pd.read_csv('/kaggle/input/hack228/coordinates.csv')
test_df = test_df.merge(coordinates_df[['postal_code', 'latitude','longitude', 'county_name', 'state_name']], on=['postal_code'], how='left')

In [ ]:
final_central_bank_df = pd.read_csv('/kaggle/input/hack228/final_central_bank.csv')
final_central_bank_df.drop(columns=['Unnamed: 0'], inplace=True)
test_df = test_df.merge(final_central_bank_df, on=['quarter'], how='left')

In [ ]:
life_expectancy_df = pd.read_csv('/kaggle/input/hack228/life_expectancy.csv')
life_expectancy_df.drop(columns=['Unnamed: 0'], inplace=True)
life_expectancy_df['year'] = life_expectancy_df['year'].astype(str)
test_df = test_df.merge(life_expectancy_df, on=['year', 'gender'], how='left')

In [ ]:
stats = ['min', 'max', 'mean', 'count', 'sum']
columns = ['npo_sum', 'npo_oprtn_grp']
cols_to_fill = [col for col in test_df.columns if '_quarters_ago' in col]
test_df[cols_to_fill] = test_df[cols_to_fill].fillna(0)

for col in tqdm(columns):
    for stat in stats:
        for i in range(1, 5):
            if i == 1:
                test_df[f'{stat}_{col}_{i}_quarters_back'] = test_df[f'{stat}_{col}_{i}_quarters_ago']
            else:
                cols_to_average = [f'{stat}_{col}_{j}_quarters_ago' for j in range(1, i+1)]
                test_df[f'{stat}_{col}_{i}_quarters_back'] = test_df[cols_to_average].mean(axis=1)

test_df.drop(columns=[*cols_to_fill], inplace=True)

In [ ]:
test_df = test_df.merge(data_account, left_on='npo_account_id', right_on='npo_accnt_id', how='left')
test_df.drop(columns=['npo_accnt_id'], inplace=True)

In [ ]:
def add_col_count_open_npo_(df):
    df_count_open_npo = df.groupby(['quarter', 'client_id'])['npo_account_id'].nunique()
    df = pd.merge(df, 
                  df_count_open_npo,
                  on=['quarter', 'client_id'],
                  suffixes=('', '_count_open'))
    return df

test_df = add_col_count_open_npo_(test_df)

In [ ]:
data_train = pd.read_parquet('/kaggle/input/hack228/data_train_fin.parquet')
col = data_train.drop(columns=['churn']).columns

In [ ]:
test_df = test_df[col]

In [37]:
test_df.to_parquet('/kaggle/working/test_df_prepared.parquet', index=False)